# 01 — Data Preparation (TMX → parallel text → train/dev/test)

**Purpose:**
 Prepare clean, aligned parallel data in the exact tab-separated format expected by NLLB and Hugging Face Datasets.

**Inputs:**

- `data/raw/source/source.txt` → Tagalog sentences.
- `data/raw/target/target.txt` → Cebuano translations.

**Process:**

- Filters short/long lines (for quality).
- Shuffles with the fixed seed for reproducibility.
- Splits into 80/10/10 train/dev/test.
- Writes to `data/processed/` as:
  - `train.tsv`
  - `dev.tsv`
  - `test.tsv`

**Outputs:**

- Aligned pairs of sentences for training and evaluation.
- Data sanity check printed via `df.sample()` (so you can visually confirm alignment).

In [1]:
import os, pathlib, sys
root = pathlib.Path(".")
print("Working directory:", root.resolve())
print("\nExpected files:")
print(" - src/prepare/convert_tmx.py")
print(" - src/prepare/make_splits.py")
print(" - data/raw/ceb-tl.tmx")

Working directory: D:\OneDrive\Documents\My Learning Resource\University Courses\DLSU\2025-26\T1\CSC715M\assignments\mc02\notebooks

Expected files:
 - src/prepare/convert_tmx.py
 - src/prepare/make_splits.py
 - data/raw/ceb-tl.tmx


## A) Convert TMX → `source.txt` & `target.txt`

This uses `lxml` with `recover=True` to tolerate broken XML.

In [2]:
!python ../src/prepare/convert_tmx.py --tmx ../data/raw/ceb-tl.tmx --src_lang tl --tgt_lang ceb --out_dir ../data/raw

Extracted 30,919 aligned pairs
Wrote:
  ..\data\raw\source\source.txt
  ..\data\raw\target\target.txt


## B) Clean & split into train/dev/test

In [3]:
!python ../src/prepare/make_splits.py --src ../data/raw/source/source.txt --tgt ../data/raw/target/target.txt --out ../data/processed


Writing splits...
  Wrote  24734 lines → ..\data\processed\train.tsv
  Wrote   3091 lines → ..\data\processed\dev.tsv
  Wrote   3093 lines → ..\data\processed\test.tsv

Verifying actual pair counts via pandas:
  train.tsv:  22851 actual pairs (pandas-parsed)
  dev.tsv:   2930 actual pairs (pandas-parsed)
  test.tsv:   2750 actual pairs (pandas-parsed)

Total usable pre-split pairs: 30,918


## C) Peek at the splits + quick QC


In [4]:
import pandas as pd, pathlib

pp = pathlib.Path("../data/processed")
for name in ["train.tsv", "dev.tsv", "test.tsv"]:
    df = pd.read_csv(pp / name, sep="\t", header=None, names=["src","tgt"])
    print(f"{name}: {len(df):,} pairs")
    display(df.sample(min(5, len(df))))

# Quick quality checks
train = pd.read_csv(pp/"train.tsv", sep="\t", header=None, names=["src","tgt"])
avg_src_len = train["src"].str.split().map(len).mean()
avg_tgt_len = train["tgt"].str.split().map(len).mean()
ratio = (train["src"].str.len() / train["tgt"].str.len()).clip(upper=10).mean()

print(f"\nAverage token length — SRC: {avg_src_len:.1f}, TGT: {avg_tgt_len:.1f}")
print(f"Average char length ratio (src/tgt, clipped): {ratio:.2f}")


train.tsv: 22,851 pairs


,src,tgt
11332,At igagawa mo ng kaniyang mga kawa upang magal...,Ug pagahimoon mo ang mga banga niini nga pagab...
12172,At aking pagkakalooban sila ng mga pananim na ...,Ug sila patindogan ko sa usa ka tanaman sa kab...
6388,Hanggang sa mabigyan ng kapahingahan ng Pangin...,Hangtud nga si Jehova magahatag sa inyong mga ...
21610,"Narito, aking ipauurong ang anino sa mga bayta...","Ania aron, pasibogon ko sa napulo ka lakang an..."
1286,At isang lalake sa mga anak ng mga propeta ay ...,Ug usa ka tawo sa mga anak nga lalake sa mga m...


dev.tsv: 2,930 pairs


,src,tgt
1015,"Anak ng tao, narito, aalisin ko sa iyo sa pama...","Anak sa tawo, ania karon, kuhaon ko gikan kani..."
318,O hinahamak mo ang mga kayamanan ng kaniyang k...,O ginatamay mo ba ang kadagaya sa iyang pagkam...
2476,"Kaya nga, sinabi niya, ang mga may kapangyarih...","Busa, matud niya, ""palugsonga didto ang inyong..."
935,"Aking minasdan ang mga bundok, at narito, nags...","Gitan-aw ko ang kabukiran, ug, ania karon, sil..."
1091,At ang prinsipe ay papasok sa daan ng portiko ...,Ug ang principe mosulod nga moagi sa alagianan...


test.tsv: 2,750 pairs


,src,tgt
799,(Ang isinaysay ng mga pantas na mula sa kanila...,(Nga gisugid sa mga tawong maalam Gikan pa sa ...
560,Siya'y tatahan sa harap ng Dios magpakailan ma...,Siya magapuyo sa atubangan sa Dios sa walay ka...
75,"At sinabi niya sa kaniya, Ganito ang sabi ng P...",Ug siya miingon kaniya: Kini mao ang gipamulon...
565,"Sa gayo'y ang mga ito ay lumabas sa bayan, ang...","Busa nanggula sa ciudad, ang mga batan-ong lal..."
2336,"Ang mga anak ni Sem; si Elam, at si Assur, at ...",Ug ang mga anak nga lalake ni Sem: si Elam ug ...



Average token length — SRC: 26.6, TGT: 32.3
Average char length ratio (src/tgt, clipped): 0.97
